In [3]:
import json

# 读取json文件
with open('/home/yuhao/THREADING-THE-NEEDLE/Dataset/Dataset_short_API.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 准备jsonl数据
jsonl_data = []
for i, item in enumerate(data):
    jsonl_entry = {
        "custom_id": f"request-{i + 1}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o-2024-08-06",
            "messages": [
                {"role": "system", "content": ""},
                {"role": "user", "content": item['prompt']}  # 将 prompt 的值作为 user 的 content
            ],
            "max_tokens": 16000,
            "stop":["*** finished"]
        }
    }
    jsonl_data.append(jsonl_entry)

# 将数据写入jsonl文件
with open('batch_api_GPT_4o.jsonl', 'w', encoding='utf-8') as f:
    for entry in jsonl_data:
        f.write(json.dumps(entry) + '\n')

print("Jsonl文件已成功保存")


Jsonl文件已成功保存


In [1]:
import json
import re

# Path to the JSONL file - Replace this with the actual file path if available
file_path = '/home/yuhao/THREADING-THE-NEEDLE/Dataset/batch_ZBdklbTPC5CeC8tZ6ytqZqZ3_output.jsonl'

# Regular expression to extract the number from 'request-1', 'request-2', etc.
pattern = re.compile(r"request-(\d+)")

# Read the file and extract numbers from custom_id in sorted order
request_numbers = []

with open(file_path, 'r') as file:
    for line in file:
        # Load the line as JSON
        data = json.loads(line)
        # Extract the custom_id
        custom_id = data.get('custom_id', '')
        # Extract the number using regex
        match = pattern.search(custom_id)
        if match:
            request_number = int(match.group(1))
            request_numbers.append(request_number)

# Check if the list of numbers is sorted from low to high
is_sorted = request_numbers == sorted(request_numbers)
request_numbers, is_sorted

([1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  

In [2]:
import json

# Path to the JSONL file
file_path = '/home/yuhao/THREADING-THE-NEEDLE/Dataset/batch_ZBdklbTPC5CeC8tZ6ytqZqZ3_output.jsonl'

# List to store the outputs
outputs = []

# Open the file and extract the outputs
with open(file_path, 'r') as file:
    for line in file:
        # Load the line as JSON
        data = json.loads(line)
        # Extract the output from the specified path in the JSON structure
        output =  data['response']['body']['choices'][0]['message']['content']
        # Append the output to the list
        outputs.append(output)

outputs[:3] # Show the first 5 outputs for verification


["#*# Week 1 (January 1st - January 7th):\nThe New Year dawned frosty and bright, with temperatures stubbornly clinging to a cool 5°C. The cold didn't deter us from our annual New Year's walk through the nearby park, a tradition that helps us set our intentions for the year ahead. Wrapped in our warmest coats, my husband and I discussed our personal and professional goals for 2018, making plans to nurture both our careers and our family. Back at work this week, the atmosphere in the biology lab was charged with fresh energy and enthusiasm. The new projects on the horizon promise challenging and innovative solutions that could redefine our approaches to environmental conservation. I spent the first few days outlining a new research project on the effects of climate change on local wildlife. At home, the post-holiday transition was smooth. We spent the evenings savoring the quiet after the festive buzz, gradually taking down the Christmas decorations. However, we decided to keep some twi

In [3]:
import argparse
import time
import json
import torch
import os


# Process output to split blocks and count words
def process_output(output: str) -> dict:
    blocks = output.split('#*#')
    word_count = len(output.split())
    return {"blocks": blocks, "word_count": word_count}

def read_json(file_path):
    with open(file_path, 'r',encoding='utf-8') as file:
        return json.load(file)
    
def write_json(file_path, data):
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

def save_to_json(data: list, filename: str) -> None:
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def load_inputs(filename: str) -> list:
    with open(filename, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

# Combine inputs, results and word counts and save them
def process_and_save_results(inputs: list, results: list, filename: str) -> None:
    combined = []
    for input_data, result_data in zip(inputs, results):
        combined.append({
            "input": input_data["prompt"],
            "checks_once": input_data["checks_once"],
            "checks_range": input_data["checks_range"],
            "checks_periodic": input_data["checks_periodic"],
            "type": input_data["type"],
            "number": input_data['number'],
            "output_blocks": result_data["blocks"],
            "word_count": result_data["word_count"]  # Adding word count here
        })
    save_to_json(combined, filename)



input_file = '/home/yuhao/THREADING-THE-NEEDLE/Dataset/Dataset_short.json'
inputs = load_inputs(input_file)

# sampling_params = SamplingParams(temperature=0.95, top_p=0.95, max_tokens=args.max_length, seed=42, repetition_penalty = 1.005)

# Setting up the LLM with the specified number of GPUs and model

output_file = './GPT_4o_16K.json'


results = [process_output(input['prefix'] + output if not output.startswith('#*#') else output) for output, input in zip(outputs, inputs)]


process_and_save_results(inputs, results, output_file)
print(f"\nSaved result to {output_file}")



Saved result to ./GPT_4o_16K.json
